In [43]:
from sklearn.cluster import MeanShift
import pandas as pd

Checkins.dat from Foursqare's dataset can be found here:
https://archive.org/details/201309_foursquare_dataset_umn
<br>
This dataset marks the checkins (visits) of users at venues. Each check-in has a unique id as well as the user id and the venue id. All data were collected in time period 2011 - 2012.

In [29]:
checkins = pd.read_csv("checkins.dat", sep='|')
che

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916,5222,NaN,NaN,2012-04-21 17:39:01
1,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391,5222,NaN,NaN,2012-04-21 17:37:18
3,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840,5222,NaN,NaN,2012-04-21 17:42:58


Remove space at the end and begining of the column names

In [32]:
checkins.columns = checkins.columns.str.strip()

Remove all rows with unrepresented latitude and longitude 

In [41]:
checkins = checkins[checkins['latitude'].notna() & checkins['longitude'].notna()]

Lets take first 100'000 rows of data and apply the MeanShift method with bandwidth = 0.1 (thats approx equal to 7 km)

In [231]:
checkins = checkins.iloc[:100000,:]
checkins = checkins[['latitude', 'longitude']]

In [158]:
model = MeanShift(bandwidth = 0.1)
predictions = model.fit_predict(checkins)

In [382]:
result = checkins.copy()
result['billNum'] = predictions

Remove all clusters with less than 16 elements

In [388]:
clusterSize = result['billNum'].value_counts().sort_index()
clusterCenters = pd.DataFrame(clusterSize)
clusterCenters['latitude'] = model.cluster_centers_[:,0]
clusterCenters['longitude'] = model.cluster_centers_[:,1]

In [390]:
clusterCenters = clusterCenters[clusterCenters['billNum'] > 15].copy()

Find distances to the offices

In [392]:
off1 = ['Los Angeles', (33.751277, -118.188740)] # (Los Angeles)
off2 = ['Miami',       (25.867736, -80.324116)]  # (Miami)
off3 = ['London',      (51.503016, -0.075479)]   # (London)
off4 = ['Amsterdam',   (52.378894, 4.885084)]    # (Amsterdam)
off5 = ['Beijing',     (39.366487, 117.036146)]  # (Beijing)
off6 = ['Sydney',      (-33.868457, 151.205134)] # (Sydney)
offices = [off1, off2, off3, off4, off5, off6]
distancesToOffices = pd.DataFrame()

In [393]:
for name, coords in offices:
    distancesToOffices[name] = ((clusterCenters['latitude'] - coords[0]) ** 2 + (clusterCenters['longitude'] - coords[1]) ** 2) ** 0.5


In [394]:
distancesToOffices['min'] = distancesToOffices.min(axis=1)
clusterCenters['minDistance'] = distancesToOffices['min']
clusterCenters.sort_values(['minDistance'], inplace = True)

Lets find 20 billboard places that is the closest to the offices

In [395]:
finalPlaces = clusterCenters.iloc[:20,:]

In [397]:
f = open('1_clustering_billboards_answer.txt','w')
f.write("{} {}".format(finalPlaces.iloc[0]['latitude'], finalPlaces.iloc[0]['longitude']))
f.close()

Final visualization of billboard distribution can be found here: https://www.mapcustomizer.com/map/Billboards%20and%20offices%20clusterization